### 测试bert使用是否正常

In [3]:
from transformers import AutoTokenizer, AutoModelForMaskedLM, BertModel
from config import *
# tokenizer = AutoTokenizer.from_pretrained("./hgmodel/bert-base-chinese/")
# model = AutoModelForMaskedLM.from_pretrained("./hgmodel/bert-base-chinese/")
# model = BertModel.from_pretrained("./hgmodel/bert-base-chinese/")
model = BertModel.from_pretrained(BERT_MODEL)


Some weights of the model checkpoint at ./hgmodel/bert-base-chinese/ were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(21128, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

### 训练测试

In [1]:
import torch

from config import *
from utils import *
from model import *

In [2]:
id2label, _ = get_label()

train_dataset = Dataset('train')
train_loader = data.DataLoader(train_dataset, batch_size=100, shuffle=True)

dev_dataset = Dataset('dev')
dev_loader = data.DataLoader(dev_dataset, batch_size=100)

model = TextCNN().to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=LR)
loss_fn = nn.CrossEntropyLoss()

dev_acc_list = []
dev_acc = 0
for e in range(10):
    for b, (input, mask, target) in enumerate(train_loader):
        input = input.to(DEVICE)
        mask = mask.to(DEVICE)
        target = target.to(DEVICE)

        pred = model(input, mask)
        loss = loss_fn(pred, target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # 每50个batch做一次
        if b % 50 != 0:
            continue
        # torch.argmax dim = 1 返回每一行中最大值的索引
        y_pred = torch.argmax(pred, dim=1)
        # pred, true, target_names=None, output_dict=False
        report =evaluate(y_pred.cpu().data.numpy(), target.cpu().data.numpy(), output_dict=True)

        with torch.no_grad():
            dev_input, dev_mask, dev_target = iter(dev_loader).__next__()
            dev_input = dev_input.to(DEVICE)
            dev_mask = dev_mask.to(DEVICE)
            dev_target = dev_target.to(DEVICE)
            dev_pred = model(dev_input, dev_mask)
            dev_pred_ = torch.argmax(dev_pred, dim=1)
            dev_report = evaluate(dev_pred_.cpu().data.numpy(), dev_target.cpu().data.numpy(), output_dict=True)
        print(
            '>> epoch:', e,
            'batch:', b,
            'loss:', round(loss.item(), 5),
            'train_acc:', report['accuracy'],
            'dev_acc:', dev_report['accuracy'],
            # type(dev_report['accuracy'])
            # type(dev_report['accuracy'])
        )
        if dev_acc < dev_report['accuracy'] and dev_report['accuracy']>0.95:
            print("-----saving model------")
            # torch.save(model, MODEL_DIR + f'{e}.pth')
            torch.save(model, MODEL_DIR + 'the_best_95.pth')
            print("-----saving model end------")
            dev_acc = dev_report['accuracy']

Some weights of the model checkpoint at ./hgmodel/bert-base-chinese/ were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


>> epoch: 0 batch: 0 loss: 2.35447 train_acc: 0.11 dev_acc: 0.22
>> epoch: 0 batch: 50 loss: 0.55928 train_acc: 0.81 dev_acc: 0.84
>> epoch: 0 batch: 100 loss: 0.25447 train_acc: 0.92 dev_acc: 0.86
>> epoch: 0 batch: 150 loss: 0.23782 train_acc: 0.93 dev_acc: 0.84
>> epoch: 0 batch: 200 loss: 0.37908 train_acc: 0.86 dev_acc: 0.88
>> epoch: 0 batch: 250 loss: 0.1411 train_acc: 0.96 dev_acc: 0.89
>> epoch: 0 batch: 300 loss: 0.33082 train_acc: 0.89 dev_acc: 0.89
>> epoch: 0 batch: 350 loss: 0.1376 train_acc: 0.97 dev_acc: 0.87
>> epoch: 0 batch: 400 loss: 0.23227 train_acc: 0.93 dev_acc: 0.89
>> epoch: 0 batch: 450 loss: 0.22827 train_acc: 0.91 dev_acc: 0.91
>> epoch: 0 batch: 500 loss: 0.22975 train_acc: 0.94 dev_acc: 0.9
>> epoch: 0 batch: 550 loss: 0.27621 train_acc: 0.92 dev_acc: 0.92
>> epoch: 0 batch: 600 loss: 0.30631 train_acc: 0.91 dev_acc: 0.91
>> epoch: 0 batch: 650 loss: 0.42344 train_acc: 0.88 dev_acc: 0.86
>> epoch: 0 batch: 700 loss: 0.45664 train_acc: 0.87 dev_acc: 0.92
>

KeyboardInterrupt: 